In [1]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix, roc_auc_score, roc_curve
from sklearn.svm import SVC
from matplotlib import colors 
from matplotlib.ticker import PercentFormatter 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Conv1D, MaxPool1D, Dropout
import os

2023-02-10 21:01:30.317893: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-10 21:01:30.479818: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 21:01:30.479839: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-10 21:01:31.043401: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
abnormal = pd.read_csv("ptbdb_abnormal.csv", header = None) 
normal = pd.read_csv("ptbdb_normal.csv", header = None)

abnormal = abnormal.drop([187], axis=1)
normal = normal.drop([187], axis=1)

abnormal['label'] = 1
normal['label'] = 0

dataset_all = pd.concat([abnormal, normal], sort=True).reset_index(drop=True)
dataset_all.head()

,0,1,2,3,4,5,6,7,8,9,...,177,178,179,180,181,182,183,184,185,186
0,0.932233,0.869679,0.886186,0.929626,0.908775,0.933970,0.801043,0.749783,0.687229,0.635100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.000000,0.606941,0.384181,0.254237,0.223567,0.276836,0.253430,0.184826,0.153349,0.121872,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.000000,0.951613,0.923963,0.853303,0.791859,0.734255,0.672043,0.685100,0.670507,0.667435,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.977819,0.899261,0.230129,0.032348,0.142329,0.223660,0.328096,0.367837,0.381701,0.389094,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.935618,0.801661,0.805815,1.000000,0.722741,0.480789,0.454829,0.319834,0.266874,0.308411,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
test_acc = []
test_recall = []
test_f1 = []
for i in range(30):
    dataset_num = dataset_all.label.value_counts().min()
    dataset_symptoms_negative = dataset_all[dataset_all['label']==0].sample(dataset_num)
    dataset_symptoms_positive = dataset_all[dataset_all['label']==1].sample(dataset_num)
    dataset_symptoms = pd.concat([dataset_symptoms_negative, dataset_symptoms_positive])

    X = dataset_symptoms.iloc[:,:-1]
    y = dataset_symptoms.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

    #Reshape train and test data to (n_samples, 187, 1), where each sample is of size (187, 1)
    X_train = np.array(X_train).reshape(X_train.shape[0], X_train.shape[1], 1)
    X_test = np.array(X_test).reshape(X_test.shape[0], X_test.shape[1], 1)

    # Create sequential model 
    clf = tf.keras.models.Sequential()

    #First CNN layer
    clf.add(Conv1D(filters=64, kernel_size=(5,), padding='same', activation=tf.keras.layers.ReLU(), input_shape = (X_train.shape[1],1)))

    #Second CNN layer
    clf.add(Conv1D(filters=64, kernel_size=(5,), padding='same', activation=tf.keras.layers.ReLU()))

    #Third CNN layer with Max pooling
    clf.add(MaxPool1D(pool_size=(5,), strides=2, padding='same'))
    clf.add(Dropout(0.5))

    #Flatten the output
    clf.add(Flatten())

    #Add a dense layer
    clf.add(Dense(units = 32, activation=tf.keras.layers.ReLU()))

    #Softmax as last layer with two outputs
    clf.add(Dense(units = 2, activation='softmax'))

    clf.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

    print(clf.summary())

    history = clf.fit(X_train, y_train, epochs=1000, batch_size = 256, validation_data = (X_test, y_test))

    y_pred1 = clf.predict(X_test)
    y_pred = np.argmax(y_pred1, axis=1)

    # Print f1, precision, and recall scores
    test_acc.append((accuracy_score(y_test, y_pred)))
    test_recall.append((recall_score(y_test, y_pred)))
    test_f1.append((f1_score(y_test, y_pred)))

    

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 187, 64)           384       
                                                                 
 conv1d_1 (Conv1D)           (None, 187, 64)           20544     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 94, 64)           0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 94, 64)            0         
                                                                 
 flatten (Flatten)           (None, 6016)              0         
                                                                 
 dense (Dense)               (None, 32)                192544    
                                                        

2023-02-10 21:01:52.151308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-10 21:01:52.151432: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-02-10 21:01:52.151524: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-02-10 21:01:52.151607: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-02-10 21:01:52.151690: W tensorfl

In [ ]:
# plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
# plt.title('model loss')
# plt.ylabel('loss')
# plt.xlabel('epoch')
# plt.legend(['train', 'val'], loc='upper left')
# plt.show()

In [ ]:
# # Predict the class probabilities
# y_probs = clf.predict(X_test)

# # Select the class with the highest predicted probability as the positive class
# y_probs = y_probs[:,1]
# y_test = y_test

# # Calculate the ROC AUC score
# roc_auc = roc_auc_score(y_test, y_probs)

# # Plot the ROC curve
# fpr, tpr, thresholds = roc_curve(y_test, y_probs)
# plt.plot(fpr, tpr, label="ROC AUC: {:.3f}".format(roc_auc))
# plt.xlabel("False Positive Rate")
# plt.ylabel("True Positive Rate")
# plt.legend()
# plt.show()

In [ ]:
# from plot_metric.functions import BinaryClassification
# # Visualisation with plot_metric
# bc = BinaryClassification(y_test, y_pred, labels=["Class 1", "Class 2"])

# # Figures
# plt.figure(figsize=(5,5))
# bc.plot_roc_curve()
# plt.show()

In [ ]:
test_recall = pd.DataFrame(test_recall, columns=['test_recall'])

test_f1 = pd.DataFrame(test_f1, columns = ['test_f1'])

test_acc = pd.DataFrame(test_acc, columns=['test_acc'])


all_events_lr = pd.concat([test_recall, test_f1, test_acc],axis=1)
all_events_lr.to_csv('n_evaluation_baseline_CNN.csv')
all_events_lr.boxplot()  
plt.ylim(0.8,1) 
plt.show()